In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__ -39.612675225  3.0317782124 0.04793662256 0.09289301524 1000.000000
accept_stat__   0.941644335  0.0831332526 0.00131445214 0.00216293273 1000.000000
   stepsize__   0.052514875  0.0047281009 0.00007475784 0.00075700743   39.009752
  treedepth__   5.835750000  0.4651086099 0.00735401283 0.01865986884  621.286025
 n_leapfrog__  69.318000000 32.0495190123 0.50674738996 0.92618871644 1000.000000
  divergent__   0.000000000  0.0000000000 0.00000000000 0.00000000000         NaN
     energy__  46.064047675  3.9459508225 0.06239096067 0.10635740956 1000.000000
        alpha   1.066128347  0.7515411900 0.01188290958 0.02630124646  816.493844
  a_society.1  -0.199779475  0.2393463065 0.00378439739 0.00530136325 1000.000000
  a_society.2   0.047279487  0.2242810695 0.00354619508 0.005667

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*